In [2]:
import pandas as pd
import spot
import os
import re

In [3]:
tools_order = ['delag', 'rabinizer4', 'ltl2tgba', 'ltl3tela', 'ltl3tela_det']

## LTL TRANSLATION

Each translator runned on 348 formulas with 60 second timeout.

In [5]:
ltl_path = '/home/ter/telatko/sat23/patterns'

In [17]:
def get_ltl_table(tools_order):
    csv = pd.read_csv(f'{ltl_path}/patterns.csv')
    ht = {
        'tool' : [],
        'ltl_success': [],
        'ltl_fail' : []
    }
    for tool in tools_order:
        #print(tool)
        tool_df = csv.loc[csv['tool'] == tool]
        success = tool_df[tool_df.exit_code == 0]
        success_count = success.shape[0]
        failed = tool_df[tool_df.exit_code != 0]
        failed_count = failed.shape[0]
        ht['tool'].append(tool)
        ht['ltl_success'].append(success_count)
        ht['ltl_fail'].append(failed_count)
        #print(f"{tool} failed: {failed_count}, successfull: {success_count}, sum: {failed_count + success_count}")
    df = pd.DataFrame(data=ht)
    return df

In [19]:
ltl_t = get_ltl_table(tools_order)
ltl_t

,tool,ltl_success,ltl_fail
0,delag,343,5
1,rabinizer4,336,12
2,ltl2tgba,348,0
3,ltl3tela,330,18
4,ltl3tela_det,328,20


## FUNKCE

In [68]:
def get_data(tools_order, path):
    data = {}
    data_small = {}

    llre = re.compile('^Running \[A([1234])\].*\'(.*?)\'$')
    hnre = re.compile('^name: "(.*)"$')
    nxre = re.compile('_([TU])\\s*$')
    

    for tool in tools_order:
        csv = pd.read_csv(f'{path}/patterns-telatko-' + tool + '.csv')
        log = f'{path}/autcross/' + tool + '.log'

        ht = {
    #        'aut_id': [],
            'L0.acc': [],
            'L1.acc': [],
            'L1.time': [],
            'L1.qbfr': [],
            'L2.acc': [],
            'L2.time': [],
            'L2.qbfr': [],
            'L3.acc': [],
            'L3.time': [],
            'L3.qbfr': [],
            'L4.acc': [],
            'L4.time': [],
            'L4.qbfr': [],
        }

        #for (ix,_),_ in fs.iterrows():
        #    ht['aut_id'].append(ix)

        i = 0
        for _,r in csv.iterrows():
            ht['L' + str(i) + '.acc'].append(r['output.acc_sets'])
            if i > 0:
                ht['L' + str(i) + '.time'].append(r['time'])

            i = (i + 1) % 5

        logfd = open(log, 'r')
        for ll in logfd.readlines():
            lmatch = llre.match(ll)
            if lmatch:
                level = lmatch.group(1)
                hoa = lmatch.group(2)
                hoa = path + "/" + hoa
               
                

                hoafd = open(hoa, 'r')
                aut_seen = False
                name_seen = False
                for hl in hoafd.readlines():
                    if len(hl) > 3:
                        aut_seen = True
                    hmatch = hnre.match(hl)
                    if hmatch:
                        name_seen = True
                        xmatch = nxre.search(hmatch.group(1))
                        if xmatch:
                            ht['L' + level + '.qbfr'].append(xmatch.group(1))
                        else:
                            ht['L' + level + '.qbfr'].append('U')
                        break

                hoafd.close()

                if not aut_seen:
                    ht['L' + level + '.qbfr'].append('X')
                elif not name_seen:
                    ht['L' + level + '.qbfr'].append('U')

        logfd.close()

        #for k in ht:
        #    print(tool, k, len(ht[k]))
        data_dd = pd.DataFrame(data = ht)
        data_de = data_dd.dropna()
        data_small[tool] = data_de[(data_de['L0.acc'] <= 1)]
        data_df = data_de[(data_de['L1.qbfr'] != 'X') & (data_de['L2.qbfr'] != 'X') & (data_de['L3.qbfr'] != 'X') & (data_de['L4.qbfr'] != 'X') & (data_de['L0.acc'] > 1)]
        data[tool] = data_df
    return data, data_small

In [69]:
def get_table(data,tools_order):
    sumdata = {
    'tool': [],
    #'ltlcount': [],
    'autcount': [],
    'L0.acc': [],
    }
    for l in range(1, 5):
        for s in ['acc', 'time', 'ratio', 'qbft', 'qbfu']:
            sumdata['L' + str(l) + '.' + s] = []

    for tool in tools_order:
        data_df = data[tool]

        sumdata['tool'].append(tool)
        sd = data_df.sum()
        #sumdata['ltlcount'].append(ltlf_translated[tool])
        sumdata['autcount'].append(len(data_df))
        for i in range(0, 5):
            sumdata['L' + str(i) + '.acc'].append(int(sd['L' + str(i) + '.acc']))
            if i > 0:
                t = sd['L' + str(i) + '.time']
                #if i > 1 and i < 4:
                #    t -= sd['L' + str(i - 1) + '.time']
                sumdata['L' + str(i) + '.time'].append(round(t, 1))
                sumdata['L' + str(i) + '.ratio'].append(
                    round(100 - 100 * int(sd['L' + str(i) + '.acc']) / int(sd['L0.acc']), 1)
                )

                for x in ['T', 'U']:
                    sumdata['L' + str(i) + '.qbf' + x.lower()].append(len(data_df[data_df['L' + str(i) + '.qbfr'] == x]))

    #for k in sumdata:
    #    print(k, len(sumdata[k]))
    r = pd.DataFrame(data = sumdata)
    return r

In [70]:
def small_aut(data_small):
    ht = {
        "tool" : [],
        "small_aut" :[],
    }

    for tool in data_small:
        df = data_small[tool]
        print(f"tool: {tool} df: {df.shape[0]}")
        ht['tool'].append(tool)
        ht['small_aut'].append(df.shape[0])
    table = pd.DataFrame(data =ht )
    return table


In [89]:
def small_aut_table(data_small):
    ht = {
        "tool" : [],
        "small_aut" : []
    }
        
    for tool in data_small:

        df = data_small[tool]
        #print(f"tool: {tool} df: {df.shape[0]}")
        ht['tool'].append(tool)
        ht['small_aut'].append(df.shape[0])
    table = pd.DataFrame(data =ht )
    return table

In [126]:
def divide_table(ltl_df, small_df, solver_data):
    count_df = pd.merge(ltl_df, small_df, on='tool')
    count_df = pd.merge(count_df, solver_data[['tool', 'autcount']])
    response_df = solver_data[['tool', 'autcount', 'L1.qbft', 'L1.qbfu', 'L2.qbft', 'L2.qbfu', 'L3.qbft', 'L3.qbfu', 'L4.qbft', 'L4.qbfu']]
    
    level_df = solver_data.drop(['L1.qbft', 'L1.qbfu', 'L2.qbft', 'L2.qbfu', 'L3.qbft', 'L3.qbfu', 'L4.qbft', 'L4.qbfu'], axis=1)
    
    return count_df, response_df, level_df

## SAT22 version

In [127]:
sat22_path = '/home/ter/telatko/sat23/SAT22_version' 

In [128]:
sat22_data, sat22_small_data = get_data(tools_order, sat22_path)

In [129]:
sat22_df = get_table(sat22_data, tools_order)
sat22_small_df = small_aut_table(sat22_small_data)

In [136]:
sat22_count_df, sat22_response_df, sat22_level_df = divide_table(ltl_t, sat22_small_df, sat22_df)

In [139]:
sat22_count_df

,tool,ltl_success,ltl_fail,small_aut,autcount
0,delag,343,5,214,129
1,rabinizer4,336,12,102,234
2,ltl2tgba,348,0,278,70
3,ltl3tela,330,18,239,91
4,ltl3tela_det,328,20,247,81


In [137]:
sat22_response_df

,tool,autcount,L1.qbft,L1.qbfu,L2.qbft,L2.qbfu,L3.qbft,L3.qbfu,L4.qbft,L4.qbfu
0,delag,129,16,113,21,108,26,103,21,108
1,rabinizer4,234,14,220,17,217,23,211,23,211
2,ltl2tgba,70,0,70,0,70,7,63,6,64
3,ltl3tela,91,12,79,15,76,18,73,18,73
4,ltl3tela_det,81,8,73,11,70,17,64,13,68


In [138]:
sat22_level_df

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,129,523,377,787.7,27.9,395,979.0,24.5,397,7231.1,24.1,373,9078.8,28.7
1,rabinizer4,234,882,552,702.8,37.4,556,938.6,37.0,564,1254.7,36.1,543,2289.7,38.4
2,ltl2tgba,70,198,198,50.8,0.0,198,78.4,0.0,189,1232.1,4.5,189,1266.0,4.5
3,ltl3tela,91,348,332,468.9,4.6,334,558.2,4.0,326,696.6,6.3,323,1551.9,7.2
4,ltl3tela_det,81,272,272,331.5,0.0,272,415.7,0.0,272,1106.6,0.0,272,1618.7,0.0


## CNF version

In [132]:
cnf_path = '/home/ter/telatko/sat23/CNF'

In [133]:
cnf_data, cnf_small_data = get_data(tools_order, cnf_path)

In [135]:
cnf_df = get_table(cnf_data, tools_order)
cnf_small_df = small_aut_table(cnf_small_data)

In [140]:
cnf_count_df, cnf_response_df, cnf_level_df = divide_table(ltl_t, cnf_small_df, cnf_df)

In [141]:
cnf_count_df

,tool,ltl_success,ltl_fail,small_aut,autcount
0,delag,343,5,214,129
1,rabinizer4,336,12,102,234
2,ltl2tgba,348,0,278,70
3,ltl3tela,330,18,239,91
4,ltl3tela_det,328,20,247,81


In [142]:
cnf_response_df

,tool,autcount,L1.qbft,L1.qbfu,L2.qbft,L2.qbfu,L3.qbft,L3.qbfu,L4.qbft,L4.qbfu
0,delag,129,17,112,21,108,26,103,22,107
1,rabinizer4,234,13,221,17,217,21,213,21,213
2,ltl2tgba,70,0,70,0,70,5,65,5,65
3,ltl3tela,91,12,79,14,77,19,72,19,72
4,ltl3tela_det,81,10,71,10,71,17,64,16,65


In [143]:
cnf_level_df

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,129,523,388,754.9,25.8,398,1041.0,23.9,397,7297.2,24.1,380,9098.7,27.3
1,rabinizer4,234,882,554,643.1,37.2,560,871.4,36.5,564,1195.0,36.1,545,2196.4,38.2
2,ltl2tgba,70,198,198,50.6,0.0,198,69.5,0.0,189,337.7,4.5,189,369.8,4.5
3,ltl3tela,91,348,337,476.7,3.2,339,554.9,2.6,330,678.4,5.2,328,1550.7,5.7
4,ltl3tela_det,81,272,272,367.8,0.0,272,411.0,0.0,272,1048.6,0.0,272,1633.0,0.0
